In [ ]:
!uv pip install -r requirements.txt

In [ ]:
from download_models import download_qwen_models

qwen_image_edit_path, qwen_image_path = download_qwen_models()

In [ ]:
from models.lightx2v_models import QwenImage

model = QwenImage(str(qwen_image_path))

In [ ]:
from pathlib import Path
import requests

def download_image(image_url: str, output_path: str, override=False):
  if Path(output_path).exists() and not override:
    return

  response = requests.get(image_url)
  response.raise_for_status()
  with open(output_path, "wb") as f:
    f.write(response.content)


In [ ]:
from PIL import Image

input_image_path_1 = "img_in_1"
output_image ="img_out.png"

download_image(
  "https://plus.unsplash.com/premium_photo-1690407617542-2f210cf20d7e?fm=jpg&q=60&w=3000&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8cGVyc29ufGVufDB8fDB8fHww",
   input_image_path_1,
)

model.generate(
  prompt="Make this person look like a robot",
  image_paths=[input_image_path_1],
  aspect_ratio="1:1",
  resolution="1K",
  output_path=output_image,
)

Image.open(output_image)

In [1]:
#!/usr/bin/env python3
import os
import re
import sys


dir_path = '/Users/sitatech/Desktop/My Hero Academia/S5 copy'

pattern = re.compile(r"ep(\d+)\.mkv$", re.IGNORECASE)

files = []
for f in os.listdir(dir_path):
    match = pattern.match(f)
    if match:
        files.append((int(match.group(1)), f))

# Rename from highest episode to lowest
for ep_num, filename in sorted(files, reverse=True):
    old_path = os.path.join(dir_path, filename)
    new_name = f"ep{ep_num + 1}.mkv"
    new_path = os.path.join(dir_path, new_name)

    if os.path.exists(new_path):
        print(f"Skipping {filename} → {new_name} (target exists)")
        continue

    os.rename(old_path, new_path)
    print(f"{filename} → {new_name}")

ep9.mkv → ep10.mkv
ep8.mkv → ep9.mkv
ep7.mkv → ep8.mkv
ep6.mkv → ep7.mkv
ep5.mkv → ep6.mkv
ep4.mkv → ep5.mkv
ep3.mkv → ep4.mkv
ep2.mkv → ep3.mkv
ep1.mkv → ep2.mkv


In [ ]:
for url in urls:
    ep_input = f"input_{count}.mkv"
    ep_output = os.path.join(output_dir, f"ep{count}.mkv")
    if os.path.exists(ep_output):
      print(f"{ep_output} already exist, skipping")
      continue